In [24]:
import pandas as pd

# Load your file
file_path = "C:\\Users\\dishi\\Downloads\\crop\\Area_Weighted_Monthly_Seasonal_And_Annual_Rainfall_0.csv"
df = pd.read_csv(file_path)

# Define how combined SD_Name entries should be split
split_mapping = {
    "ASSAM & MEGHALAYA": ["ASSAM", "MEGHALAYA"],
    "HARYANA, DELHI & CHANDIGARH": ["HARYANA", "DELHI", "CHANDIGARH"],
    "NAGALAND, MANIPUR, MIZORAM,TRIPURA": ["NAGALAND", "MANIPUR", "MIZORAM", "TRIPURA"],
    "SUB-HIMALAYAN W BENGAL & SIKKIM": ["WEST BENGAL", "SIKKIM"],
    "TAMIL NADU & PONDICHERRY": ["TAMIL NADU", "PUDUCHERRY"],
    "GUJARAT REGION, DADRA & NAGAR HAVELI": ["GUJARAT", "DADRA & NAGAR HAVELI"],
    "SAURASHTRA KUTCH & DIU": ["GUJARAT", "DIU"]
}

# Expand rows in-place and update df directly
expanded_rows = []
for _, row in df.iterrows():
    sd_value = str(row["SD_Name"]).strip().upper()
    if sd_value in split_mapping:
        for new_sd in split_mapping[sd_value]:
            new_row = row.copy()
            new_row["SD_Name"] = new_sd
            expanded_rows.append(new_row)
    else:
        expanded_rows.append(row)

# Replace df with the expanded data
df = pd.DataFrame(expanded_rows)

# Check result summary
print("✅ Updated in-place:")
print("Total rows:", df.shape[0])
print("Unique SD_Name count:", df['SD_Name'].nunique())

# (Optional) If you want to save changes:
# df.to_csv(file_path, index=False)


✅ Updated in-place:
Total rows: 2941
Unique SD_Name count: 45


In [25]:

# --- 2️⃣ Clean SD_Name formatting ---
df["SD_Name"] = (
    df["SD_Name"]
    .str.upper()
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# --- 3️⃣ Define east-west merge map with weights ---
merge_map = {
    "EAST RAJASTHAN": ("RAJASTHAN", 0.7),
    "WEST RAJASTHAN": ("RAJASTHAN", 0.3),
    "EAST MADHYA PRADESH": ("MADHYA PRADESH", 0.6),
    "WEST MADHYA PRADESH": ("MADHYA PRADESH", 0.4),
    "EAST UTTAR PRADESH": ("UTTAR PRADESH", 0.6),
    "WEST UTTAR PRADESH": ("UTTAR PRADESH", 0.4),
}

# --- 4️⃣ Map unified state names + weights ---
df["STATE"] = df["SD_Name"].apply(lambda x: merge_map.get(x, (x, 1))[0])
df["WEIGHT"] = df["SD_Name"].apply(lambda x: merge_map.get(x, (x, 1))[1])

# --- 5️⃣ Perform weighted aggregation (keep numeric columns only) ---
numeric_cols = df.select_dtypes(include=["number"]).columns.difference(["WEIGHT"])
grouped = (
    df.groupby(["YEAR", "STATE"], as_index=False)
    .apply(lambda g: pd.Series({
        col: (g[col] * g["WEIGHT"]).sum() / g["WEIGHT"].sum() if col in numeric_cols else None
        for col in numeric_cols
    }))
    .reset_index(drop=True)
)

# --- 6️⃣ Replace old DataFrame with merged version ---
df = grouped.copy()
df.rename(columns={"STATE": "SD_Name"}, inplace=True)

# ✅ In-memory update complete
print("✅ DataFrame updated: East/West subdivisions merged into unified states.")
print("Unique SD_Name values now:", sorted(df["SD_Name"].unique()))


✅ DataFrame updated: East/West subdivisions merged into unified states.
Unique SD_Name values now: ['ANDAMAN & NICOBAR ISLANDS', 'ARUNACHAL PRADESH', 'ASSAM', 'BIHAR', 'CHANDIGARH', 'CHATTISGARH', 'COASTAL KARNATAKA', 'COSTAL ANDHRA PRADESH', 'DADRA & NAGAR HAVELI', 'DELHI', 'DIU', 'GANGETIC WEST BENGAL', 'GUJARAT', 'HARYANA', 'HIMACHAL PRADESH', 'JAMMU & KASHMIR', 'JHARKHAND', 'KERALA', 'KOKAN & GOA', 'LAKSHADWEEP', 'MADHYA MAHARASHTRA', 'MADHYA PRADESH', 'MANIPUR', 'MARATWADA', 'MEGHALAYA', 'MIZORAM', 'NAGALAND', 'NORTH INTERIOR KARNATAKA', 'ORISSA', 'PUDUCHERRY', 'PUNJAB', 'RAJASTHAN', 'RAYALSEEMA', 'SIKKIM', 'SOUTH INTERIOR KARNATAKA', 'TAMIL NADU', 'TELENGANA', 'TRIPURA', 'UTTAR PRADESH', 'UTTARANCHAL', 'VIDARBHA', 'WEST BENGAL']


C:\Users\dishi\AppData\Local\Temp\ipykernel_19164\4033221041.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [28]:


# --- Step 1: Define the IMD → State mapping ---
imd_to_state = {
    'ANDAMAN & NICOBAR ISLANDS': 'Andaman and Nicobar Islands',
    'ARUNACHAL PRADESH': 'Arunachal Pradesh',
    'ASSAM': 'Assam',
    'MEGHALAYA': 'Meghalaya',
    'NAGALAND': 'Nagaland',
    'MANIPUR': 'Manipur',
    'MIZORAM': 'Mizoram',
    'TRIPURA': 'Tripura',
    'GANGETIC WEST BENGAL': 'West Bengal',
    'WEST BENGAL': 'West Bengal',
    'SIKKIM': 'Sikkim',
    'ORISSA': 'Odisha',
    'JHARKHAND': 'Jharkhand',
    'BIHAR': 'Bihar',
    'UTTAR PRADESH': 'Uttar Pradesh',
    'UTTARANCHAL': 'Uttarakhand',
    'HARYANA': 'Haryana',
    'CHANDIGARH': 'Chandigarh',
    'PUNJAB': 'Punjab',
    'HIMACHAL PRADESH': 'Himachal Pradesh',
    'JAMMU & KASHMIR': 'Jammu and Kashmir',
    'DELHI': 'Delhi',
    'RAJASTHAN': 'Rajasthan',
    'GUJARAT': 'Gujarat',
    'KOKAN & GOA': 'Goa',
    'MADHYA MAHARASHTRA': 'Maharashtra',
    'MARATWADA': 'Maharashtra',
    'VIDARBHA': 'Maharashtra',
    'CHATTISGARH': 'Chhattisgarh',
    'MADHYA PRADESH': 'Madhya Pradesh',
    'COASTAL KARNATAKA': 'Karnataka',
    'NORTH INTERIOR KARNATAKA': 'Karnataka',
    'SOUTH INTERIOR KARNATAKA': 'Karnataka',
    'TELENGANA': 'Telangana',
    'RAYALSEEMA': 'Andhra Pradesh',
    'COSTAL ANDHRA PRADESH': 'Andhra Pradesh',
    'TAMIL NADU': 'Tamil Nadu',
    'KERALA': 'Kerala',
    'LAKSHADWEEP': 'Lakshadweep',
    'DADRA & NAGAR HAVELI': 'Dadra and Nagar Haveli',
    'DIU': 'Daman and Diu',
    'PUDUCHERRY': 'Puducherry'
}

# --- Step 2: Map IMD Subdivision to State ---
df["State"] = df["SD_Name"].map(imd_to_state)

# --- Step 3: Check unmapped values ---
unmapped = df[df["State"].isna()]["SD_Name"].unique()
if len(unmapped) > 0:
    print("⚠️ Unmapped Subdivisions:", unmapped)
else:
    print("✅ All subdivisions mapped successfully.")

# --- Step 4: Weighted mean aggregation ---
# Assuming you have columns: 'Year', 'Rainfall', and 'Weight' (e.g. area or population)
# If you don’t have a 'Weight' column, create uniform weights = 1
if 'Weight' not in df.columns:
    df["Weight"] = 1

# Weighted mean formula: sum(value * weight) / sum(weight)
df_grouped = (
    df.groupby(["YEAR", "State"], as_index=False)
      .apply(lambda g: pd.Series({
          **{col: (g[col] * g["Weight"]).sum() / g["Weight"].sum()
             for col in g.columns if col not in ["Year", "State", "SD_Name", "Weight"] and pd.api.types.is_numeric_dtype(g[col])},
          "Weight": g["Weight"].sum()
      }))
      .reset_index(drop=True)
)

# --- Step 5: Replace df with the new grouped version ---
df = df_grouped

print("✅ Duplicates merged — weighted mean computed for repeating Year+State pairs.")


✅ All subdivisions mapped successfully.
✅ Duplicates merged — weighted mean computed for repeating Year+State pairs.


C:\Users\dishi\AppData\Local\Temp\ipykernel_19164\884629223.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [30]:
# Get unique State values
unique_state_names = df['State'].unique()

print(f"Unique State values ({len(unique_state_names)}):")
print("="*80)
for name in sorted(unique_state_names):
    print(f"  - {name}")

Unique State values (36):
  - Andaman and Nicobar Islands
  - Andhra Pradesh
  - Arunachal Pradesh
  - Assam
  - Bihar
  - Chandigarh
  - Chhattisgarh
  - Dadra and Nagar Haveli
  - Daman and Diu
  - Delhi
  - Goa
  - Gujarat
  - Haryana
  - Himachal Pradesh
  - Jammu and Kashmir
  - Jharkhand
  - Karnataka
  - Kerala
  - Lakshadweep
  - Madhya Pradesh
  - Maharashtra
  - Manipur
  - Meghalaya
  - Mizoram
  - Nagaland
  - Odisha
  - Puducherry
  - Punjab
  - Rajasthan
  - Sikkim
  - Tamil Nadu
  - Telangana
  - Tripura
  - Uttar Pradesh
  - Uttarakhand
  - West Bengal


In [33]:
# Filter data to keep only years between 2005 and 2014 (inclusive)
df = df[(df["YEAR"] >= 2005) & (df["YEAR"] <= 2014)]

print(f"✅ Filtered dataframe: {len(df)} rows remain (2005–2014 only)")


✅ Filtered dataframe: 360 rows remain (2005–2014 only)


In [35]:
# Export final dataframe to CSV
df.to_csv("rain.csv", index=False, encoding="utf-8-sig")

print("✅ DataFrame successfully saved as 'rain.csv'")


✅ DataFrame successfully saved as 'rain.csv'


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 360 entries, 1941 to 2300
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   State    360 non-null    object 
 1   ANNUAL   360 non-null    float64
 2   APR      360 non-null    float64
 3   AUG      360 non-null    float64
 4   DEC      360 non-null    float64
 5   FEB      360 non-null    float64
 6   JAN      360 non-null    float64
 7   JAN-FEB  360 non-null    float64
 8   JUL      360 non-null    float64
 9   JUN      360 non-null    float64
 10  Jun-Sep  360 non-null    float64
 11  MAR      360 non-null    float64
 12  MAY      360 non-null    float64
 13  Mar-May  360 non-null    float64
 14  NOV      360 non-null    float64
 15  OCT      360 non-null    float64
 16  Oct-Dec  360 non-null    float64
 17  SEP      360 non-null    float64
 18  Year     360 non-null    int64  
dtypes: float64(17), int64(1), object(1)
memory usage: 56.2+ KB


In [39]:
# Rename YEAR column to Year
df.rename(columns={'YEAR': 'Year'}, inplace=True)
print("✅ Column renamed: 'YEAR' → 'Year'")



✅ Column renamed: 'YEAR' → 'Year'


C:\Users\dishi\AppData\Local\Temp\ipykernel_19164\895440806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'YEAR': 'Year'}, inplace=True)


In [42]:
# Convert Year column from float64 to int
df['Year'] = df['Year'].astype(int)

print("✅ Year column converted to int")
print(f"\nData type: {df['Year'].dtype}")
print(f"Sample values: {df['Year'].unique()[:10]}")

df.to_csv('rain.csv', index=False)

✅ Year column converted to int

Data type: int64
Sample values: [2005 2006 2007 2008 2009 2010 2011 2012 2013 2014]


C:\Users\dishi\AppData\Local\Temp\ipykernel_19164\3665913456.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['Year'].astype(int)


In [46]:
# Drop Weight and SD NO. columns
print("Columns before dropping:")
print(list(df.columns))
print(f"\nTotal columns: {len(df.columns)}")

# Drop the columns
df = df.drop(columns=['Weight', 'SD NO.','Oct-Dec','Mar-May','Jun-Sep','JAN-FEB'], errors='ignore')

print("\n" + "="*80)
print("\nColumns after dropping:")
print(list(df.columns))
print(f"\nTotal columns: {len(df.columns)}")

print("\n✅ Dropped columns: Weight, SD NO.")

# Save updated dataframe
df.to_csv('rain.csv', index=False)
print("✅ Updated rain.csv saved")



Columns before dropping:
['State', 'ANNUAL', 'APR', 'AUG', 'DEC', 'FEB', 'JAN', 'JAN-FEB', 'JUL', 'JUN', 'Jun-Sep', 'MAR', 'MAY', 'Mar-May', 'NOV', 'OCT', 'Oct-Dec', 'SEP', 'Year']

Total columns: 19


Columns after dropping:
['State', 'ANNUAL', 'APR', 'AUG', 'DEC', 'FEB', 'JAN', 'JUL', 'JUN', 'MAR', 'MAY', 'NOV', 'OCT', 'SEP', 'Year']

Total columns: 15

✅ Dropped columns: Weight, SD NO.
✅ Updated rain.csv saved
